# Welcome to Projects in Programming & Data Science! 

# Setup

## Brightspace

Make sure you are able to access our course page on __Brightspace__. This is where I will be posting all of our course content (notebooks, datasets, homeworks, et. cetera). 

__This is also where all of your assignments will be posted, turned in, graded, and returned.__

## Using Colab

In order to standardize the way we all code, we are going to use [Google's Colab](https://colab.research.google.com/notebooks/welcome.ipynb).

Think of using Colab as renting a computer via your web browser (I recommend Chrome) that you can use.

For instance, if I want to open today's class notebook in Colab, I: 

<br>

1. *Will go to https://colab.sandbox.google.com*

2. *Will click "Upload" then "Choose File"* 

3. *Note that a Jupyter Notebook will always have a .ipynb extension. In Colab I can simply upload the notebook (that I have downloaded from Brightspace) to Colab.*

4. *Once I click "Open" Colab should open the notebook, and voila!*

# Using Colab

## Cell Types

There are two types of cells that you can use: "Text" and "Code. 

*   _Text:_ Allows you to use [markdown](https://www.markdownguide.org/cheat-sheet/).
*   _Code:_ Allows you to type code.


For example, below is the same code in a text cell and then again in a code cell: 

In [ ]:
# Below is some code. Note that this is in a code cell, so in order to write text (we call this "commenting" I need to use the pound sign to "comment out" the text)

print("Hello, World!")

In [ ]:
If I try to write text in a code cell without commenting it out, I will get an error...

## Saving

Colab will automatically save changes. Under the "File" tab at the top-left of the window, you also have the option to save a copy to Google Drive or GitHub, and to download directly to your machine. 

## Collaborating

You can easily share a notebook by clicking on "Share" at the top-right of the window and adding the email of your collaborator. 

## Commenting

If you want to leave comments or action items for collaborators, you can use the "Comment" option at the top-right of the window, just as you would in Google Docs or Word. 

## "Running" A Cell

In order to run (or, 'execute') a cell, you can hold down the shift key and hit "return" or "enter". You can also view additional options under the "Runtime" tab at the top-left of the window.

---

# ⭕ **QUESTIONS?**

---

# Jumping In

Today we're going to jump right in to the mix and leverage the CitiBike API to populate a sqlite database at regular intervals. Consider this your warm-up for the semester! 

In [ ]:
import sqlite3 # this is how we will import the sqlite3 functionality needed to proceed

[SQLite](https://www.sqlite.org/index.html) is a library that allows us to create, populate, and call upon a SQL Database. It's also serverless, meaning we don't need to access a separate server where we're storing our data – instead, we can directly access our databse. We can even store that database as a file in our Colab environment and call upon it. 

In [ ]:
con = sqlite3.connect('citibikeData2022+class1.db') # this is how we are going to create our database, 
                                         # calling it 'citibikeData2022+class1.db'

                                         # note that if the db doesn't exist, this will create it; Otherwise, it will connect

# "con" stands for "connection" – this is telling SQLite what database to use


It's important to note that the databse we just created was created _inside this Colab environment_ which will be reset when we log off. 

Thus, when you're done with your databse, I recommend downloading it directly to your machine. 

You can see the databse by clicking on the file icon on the far-left side of the window. 

---

# ⭕ **QUESTIONS?**

---

Let's check out the API we'll be working with: http://gbfs.citibikenyc.com/gbfs/gbfs.json

First, we'll request the first json from the CitiBike API URL and just print it out to get a quick glimpse

In [ ]:
import json 
import urllib.request # https://docs.python.org/3/library/urllib.request.html

with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_information.json") as url:
    station_info = json.loads(url.read().decode())
    print(station_info)

As you can see, the json is a dictionary of lists and other dictionaries containing information about CitiBike stations across New York City. 

In [ ]:
for key,value in station_info.items(): # we begin by unpacking this dictionary
    print("key: {0} | value: {1}".format(key, value))

In [ ]:
station_info_values_list = list(station_info.values())

In [ ]:
station_info_values_list

In [ ]:
clean_station_info = station_info_values_list[0]['stations']

In [ ]:
clean_station_info[0]

Now let's do the same for the second API that contains some additional information on these stations.

In [ ]:
import json 
import urllib.request # https://docs.python.org/3/library/urllib.request.html

with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_status.json") as url:
    station_status = json.loads(url.read().decode())
    print(station_status)

In [ ]:
for key,value in station_status.items():
    print("key: {0} | value: {1}".format(key, value))

In [ ]:
station_status_values_list = list(station_status.values())

In [ ]:
station_status_values_list[0]

In [ ]:
clean_station_status = station_status_values_list[0]['stations']

In [ ]:
clean_station_status[0]

---

# ⭕ **QUESTIONS?**

---

In [ ]:
import pandas as pd # we'll use pandas just to visualize our data, NOT to query it

df_station_info = pd.DataFrame(clean_station_info) # create a new dataframe called 'df_stations' 
df_station_info.head() # check the first five station entries

In [ ]:
import pandas as pd # we'll use pandas just to visualize our data, NOT to query it

df_station_status = pd.DataFrame(clean_station_status) # create a new dataframe called 'df_stations' 
df_station_status.head() # check the first five station entries

---

# ⭕ **QUESTIONS?**

---

So, we have our data from the CitiBike feed, and it looks pretty good! Now we need to create two tables within our database (the one we named citibikeData.db). We do that using the 'CREATE TABLE IF NOT EXISTS' statement seen below. 

In that statement, the 'IF NOT EXISTS' makes clear that we are going to create the table called 'StationsData' only once. That way, if we run that cell again, it's not going to overwrite the work we've previously done. 

Note that at this point we aren't adding any data to our table. All we're doing is telling SQLite that we want to create a new table, and providing it with a) the column names and b) the data type those columns should be expecting.

In [ ]:
sql = "CREATE TABLE IF NOT EXISTS AllStationData (capacity INTEGER, lat BLOB, lon BLOB, name TEXT, region_id INTEGER, station_type TEXT, is_installed INTEGER, is_renting INTEGER, is_returning INTEGER, num_bikes_available INTEGER, num_bikes_disabled INTEGER, num_docks_available INTEGER, num_docks_disabled INTEGER, num_ebikes_available INTEGER, station_id INTEGER, station_status TEXT );" 

con.execute(sql)
con.commit()

Now that we have our database and our table, we want to insert our data. 

Below, we create a "query template" where we "INSERT OR IGNORE INTO" our table the values associated with each of our columns. 

We define those values by parsing through the CitiBike json we got earlier, and for each "row" of that json, we create a new row in our SQLite table. 

First, we have to do some complicated work with 'zips'...

In [ ]:
list1 = [0,2,4,6,8,10]
list2 = [1,3,5,7,9]

for a,b in zip(list1, list2): 
  print("a: ", a , "; b: ", b)

In [ ]:
zipped = zip(list1,list2)

list(zipped)

In [ ]:
AllStationData = zip(clean_station_info, clean_station_status)

list(AllStationData)[0]

In [ ]:
for info, status in zip(clean_station_info,clean_station_status):
  print(info,status)
  break

In [ ]:
for info, status in zip(clean_station_info,clean_station_status):
  print(info['lon'],status['is_installed'])
  break

In [ ]:
query_template = """INSERT OR IGNORE INTO AllStationData (capacity, lat, lon, name, station_type, is_installed, is_renting, is_returning, num_bikes_available, num_bikes_disabled, num_docks_available, num_docks_disabled, num_ebikes_available, station_id, station_status)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"""

for info, status in zip(clean_station_info,clean_station_status):

      capacity = info['capacity']
      lat = info['lat']
      lon = info['lon']
      name = info['name']
      station_type = info['station_type']
      is_installed = status['is_installed']
      is_renting = status['is_renting'] 
      is_returning = status['is_returning']
      num_bikes_available = status['num_bikes_available'] 
      num_bikes_disabled = status['num_bikes_disabled']
      num_docks_available = status['num_docks_available']
      num_docks_disabled = status['num_docks_disabled'] 
      num_ebikes_available = status['num_ebikes_available']
      station_id = info['station_id']
      station_status = status['station_status']
                           
      print("Inserting Station:", capacity, lat, lon, name, station_type, is_installed, is_renting, is_returning, num_bikes_available, num_bikes_disabled, num_docks_available, num_docks_disabled, num_ebikes_available, station_id, station_status) 

      query_parameters = (capacity, lat, lon, name, station_type,is_installed, is_renting, is_returning, num_bikes_available, num_bikes_disabled, num_docks_available, num_docks_disabled, num_ebikes_available, station_id, station_status) 
    
      con.execute(query_template, query_parameters)
    
con.commit()

Now, we can use [pd.read_sql](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_sql.html) to check that we are properly connected to our database, and the StationsData table within that database:

In [ ]:
check = pd.read_sql("SELECT * FROM AllStationData LIMIT 5", con=con)
check

---

# ⭕ **QUESTIONS?**

---

Looks good! Last but not least, let's set things up so that our database automatically updates every 15 seconds. 

In [ ]:
import time 
import json 
import urllib.request
from datetime import datetime

while True:
  
    with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_information.json") as url:
        station_info = json.loads(url.read().decode())
        station_info_values_list = list(station_info.values())
        clean_station_info = station_info_values_list[0]['stations']

    with urllib.request.urlopen("https://gbfs.citibikenyc.com/gbfs/en/station_status.json") as url:
        station_status = json.loads(url.read().decode())
        station_status_values_list = list(station_status.values())
        clean_station_status = station_status_values_list[0]['stations']
    
    query_template = """INSERT OR IGNORE INTO AllStationData (capacity, lat, lon, name, station_type, is_installed, is_renting, is_returning, num_bikes_available, num_bikes_disabled, num_docks_available, num_docks_disabled, num_ebikes_available, station_id, station_status)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);"""

    for info, status in zip(clean_station_info,clean_station_status):

      capacity = info['capacity']
      lat = info['lat']
      lon = info['lon']
      name = info['name']
      station_type = info['station_type']
      is_installed = status['is_installed']
      is_renting = status['is_renting'] 
      is_returning = status['is_returning']
      num_bikes_available = status['num_bikes_available'] 
      num_bikes_disabled = status['num_bikes_disabled']
      num_docks_available = status['num_docks_available']
      num_docks_disabled = status['num_docks_disabled'] 
      num_ebikes_available = status['num_ebikes_available']
      station_id = info['station_id']
      station_status = status['station_status']
                           
      print("Inserting Station:", capacity, lat, lon, name, station_type, is_installed, is_renting, is_returning, num_bikes_available, num_bikes_disabled, num_docks_available, num_docks_disabled, num_ebikes_available, station_id, station_status) 

      query_parameters = (capacity, lat, lon, name, station_type,is_installed, is_renting, is_returning, num_bikes_available, num_bikes_disabled, num_docks_available, num_docks_disabled, num_ebikes_available, station_id, station_status) 
    
      con.execute(query_template, query_parameters)
    
    con.commit()
        
time.sleep(15)

---

I hope this has helped you find your programming legs! Next week we'll get back to descriptive analytics using Python and Pandas. For now, take time to refresh yourself on the content covered in "Introduction to Programming". 